In [ ]:


from constants.abs_paths import AbsDirPath
import pandas as pd

from processing_pipeline.utilities.data_transformation import split_dataset_by_repo_and_source, load_all_files

In [ ]:
file_dir = AbsDirPath.KEYWORDS_MATCHING

df = load_all_files(file_dir)

In [ ]:
df.head()

In [ ]:
df.shape

## Reduce dataset size based on lowercase matched keywords
Preserve original matched words and count of matches...

In [ ]:
df.groupby("matched_word").size()

In [ ]:
df["matched_word_lw"] = df.matched_word.str.lower()

In [ ]:
df.groupby("matched_word_lw").size()

In [ ]:
matched_word_df = df.groupby("matched_word_lw").matched_word.agg(unique_matched_words=("unique"), nunique_matched_words=("nunique"))
matched_word_df

In [ ]:
df = pd.merge(df, matched_word_df, "left", "matched_word_lw")
df

In [ ]:
df = df.drop(columns="matched_word")
df = df.rename(columns={"matched_word_lw": "matched_word"})
df

## Clean sentences

In [ ]:
df.sentence = df.sentence.str.replace(r"((\t|\r) ?)+", " ", regex=True)

In [ ]:
df.sentence = df.sentence.str.replace(r"[ \"'#]{2,}", " ", regex=True)

## Assign IDs to similar items
Similar items are considered the ones that have all the similar parameters but the match idx

Also remove match idx, as it is not used

In [ ]:
df = df.drop(columns="match_idx")

In [ ]:
import hashlib

quick_hash = lambda x: hashlib.md5(x.encode('utf-8')).hexdigest()

df["sentence_hash"] = df['sentence'].apply(quick_hash)

In [ ]:
df.head()

In [ ]:
df['id'] = df.repo_id + "." + df.source + "." + df.qa + "." +  df.sentence_hash + "." + df.keyword + "." +df.matched_word
df.head()

In [ ]:
df = df.drop(columns=["sentence_hash"])

In [ ]:
df["nsimilar"] = df.groupby("id").transform("size")
df = df.groupby("id").first()
df.head()

### Matched word clean up results: before and after

In [ ]:
df.groupby("keyword").nsimilar.agg(["sum", "count"]).sum(axis=0)

In [ ]:
df.groupby("keyword").nsimilar.agg(["sum", "count"]).sum(axis=0)

In [ ]:
df.groupby("keyword").nsimilar.agg(["sum", "count"])

## Output to optimized folder

In [ ]:
df.head()

In [ ]:
# avoid inconsistent read issue with parquet and pyarrow
df.unique_matched_words = df.unique_matched_words.agg(", ".join)
df.head()

In [ ]:
output_dir = AbsDirPath.KEYWORDS / "matched_o"

output_dir.mkdir(exist_ok=True)
split_dataset_by_repo_and_source(output_dir, df.reset_index())


In [ ]:
from utilities.split_parquet import split_files_exceeding_max_limit

split_files_exceeding_max_limit(output_dir)